In [183]:
#!pip install pandas
import pandas as pd
#!pip install numpy
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

# DATA PREP

Uploading the data

In [186]:
df=pd.read_csv(r".csv")

Types & number of features and rows

In [188]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 168639 entries, 0 to 168638
Data columns (total 29 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    168639 non-null  int64  
 1   name                  168634 non-null  object 
 2   number_of_seasons     168639 non-null  int64  
 3   number_of_episodes    168639 non-null  int64  
 4   original_language     168639 non-null  object 
 5   vote_count            168639 non-null  int64  
 6   vote_average          168639 non-null  float64
 7   overview              93333 non-null   object 
 8   adult                 168639 non-null  bool   
 9   backdrop_path         77780 non-null   object 
 10  first_air_date        136903 non-null  object 
 11  last_air_date         138735 non-null  object 
 12  homepage              50998 non-null   object 
 13  in_production         168639 non-null  bool   
 14  original_name         168634 non-null  object 
 15  

Filtering "first_air_date" field between 2018-2023

In [190]:
# Converting first_air_date for object to datetime
df["first_air_date"] = pd.to_datetime(df["first_air_date"], errors="coerce")

# Filtering rows where the year is between 2018 and 2023
df2 = df[(df["first_air_date"].dt.year >= 2018) & (df["first_air_date"].dt.year <= 2023)]

# Converting the 'first_air_date' column back to string type
df2["first_air_date"] = df2["first_air_date"].astype("string")

In [191]:
# Founding in the data duplicate rows (by id) with same/very similar info but with different target value (popularity) - decided to remove
# Remove all rows where 'id' appears more than once
df2 = df2[df2.groupby("id")["id"].transform("count") == 1]

Dropping irrelevant features

In [193]:
# More than 55% of nulls, duplicate features, id, URLs
columns_to_drop = ["backdrop_path", "homepage", "poster_path", "tagline", "created_by", "production_companies", "spoken_languages", "original_language", "id", "original_name", "in_production", "production_countries"]
df2 = df2.drop(columns=columns_to_drop)

Value counts of the categorial features

In [195]:
# Handling netwroks feature

# Splitting the 'networks' feature by commas, only process if it's a string
df2['networks'] = df2['networks'].apply(lambda x: x.split(',') if isinstance(x, str) else x)

# Removing leading/trailing spaces from each network name and clean the network names
df2['networks'] = df2['networks'].apply(lambda x: [network.strip().lower().replace('-', '') for network in x] if isinstance(x, list) else x)

# Flattening the list of networks, getting the frequency count (counting each network)
network_counts = df2['networks'].explode().value_counts()

# Geting the top 10 networks
top_10_networks = network_counts.head(10).index

# Replaceing networks not in the top 10 with 'Other' in the original column, but keep NaN as NaN
def group_networks(networks):
    if isinstance(networks, list):  # Only process if it's a list (after split)
        return [network if network in top_10_networks else 'Other' for network in networks]
    else:  # Keeping NaN values unchanged
        return networks

df2['networks'] = df2['networks'].apply(group_networks)

# Getting the frequency count and percentage of the grouped networks
grouped_network_counts = df2['networks'].explode().value_counts()
grouped_network_percentage = (grouped_network_counts / len(df2)) * 100

# Combining the counts and percentages into a DataFrame
grouped_network_summary = pd.DataFrame({
    'Count': grouped_network_counts,
    'Percentage': grouped_network_percentage
})

# Sorting by count and displaying the result
grouped_network_summary = grouped_network_summary.sort_values(by='Count', ascending=False)

# Printing the results
print("\nGrouped Networks (Top 10 + 'Other') Frequency and Percentage:")
print(grouped_network_summary)

# Displaying the updated df2 with the modified 'networks' column
print("\nUpdated df2 with Grouped Networks:")
print(df2[['networks']].head())


Grouped Networks (Top 10 + 'Other') Frequency and Percentage:
               Count  Percentage
networks                        
Other          31933   62.160321
netflix         1523    2.964650
youtube         1383    2.692128
iqiyi            774    1.506657
tencent video    728    1.417114
prime video      631    1.228296
youku            489    0.951880
tokyo mx         408    0.794207
discovery+       381    0.741649
channel 4        356    0.692985
channel 5        354    0.689091

Updated df2 with Grouped Networks:
     networks
6   [netflix]
9     [Other]
12    [Other]
14    [Other]
16    [Other]


In [196]:
# Handling languages feature

# Splitting the 'languages' feature by commas, only process if it's a string
df2['languages'] = df2['languages'].apply(lambda x: x.split(',') if isinstance(x, str) else x)

# Removing leading/trailing spaces from each network name and clean the network names
df2['languages'] = df2['languages'].apply(lambda x: [language.strip().lower().replace('-', '') for language in x] if isinstance(x, list) else x)

# Flattening the list of language, getting the frequency count (counting each language)
language_counts = df2['languages'].explode().value_counts()

# Geting the top 10 languages
top_10_languages = language_counts.head(10).index

# Replaceing networks not in the top 10 with 'Other' in the original column, but keep NaN as NaN
def group_languages(languages):
    if isinstance(languages, list):  # Only process if it's a list (after split)
        return [language if language in top_10_languages else 'Other' for language in languages]
    else:  # Keep NaN values unchanged
        return languages

df2['languages'] = df2['languages'].apply(group_languages)

# Getting the frequency count and percentage of the grouped language
grouped_language_counts = df2['languages'].explode().value_counts()
grouped_language_percentage = (grouped_language_counts / len(df2)) * 100

# Combining the counts and percentages into a DataFrame
grouped_language_summary = pd.DataFrame({
    'Count': grouped_language_counts,
    'Percentage': grouped_language_percentage
})

# Sorting by count and displaying the result
grouped_language_summary = grouped_language_summary.sort_values(by='Count', ascending=False)

# Printing the results
print("\nGrouped Languages (Top 10 + 'Other') Frequency and Percentage:")
print(grouped_language_summary)

# Displaying the updated df2 with the modified 'language' column
print("\nUpdated df2 with Grouped Languages:")
print(df2[['languages']].head())


Grouped Languages (Top 10 + 'Other') Frequency and Percentage:
           Count  Percentage
languages                   
en         10381   20.207506
Other       9817   19.109632
zh          3904    7.599471
ja          3343    6.507436
fr          2645    5.148719
ko          2409    4.689325
de          1733    3.373433
es          1671    3.252745
nl          1391    2.707701
pt          1237    2.407926
hi          1168    2.273612

Updated df2 with Grouped Languages:
          languages
6   [en, ko, Other]
9              [en]
12             [en]
14             [en]
16             [en]


In [197]:
# Handling "genres" column 

# Check for unique genres
unique_genres = set(genre for genres in df2['genres'] if isinstance(genres, list) for genre in genres)

print(f"Total unique genres: {len(unique_genres)}")
print(unique_genres)

Total unique genres: 0
set()


In [198]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51372 entries, 6 to 168636
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   name                51372 non-null  object 
 1   number_of_seasons   51372 non-null  int64  
 2   number_of_episodes  51372 non-null  int64  
 3   vote_count          51372 non-null  int64  
 4   vote_average        51372 non-null  float64
 5   overview            29404 non-null  object 
 6   adult               51372 non-null  bool   
 7   first_air_date      51372 non-null  string 
 8   last_air_date       50938 non-null  object 
 9   popularity          51372 non-null  float64
 10  type                51372 non-null  object 
 11  status              51372 non-null  object 
 12  genres              34485 non-null  object 
 13  languages           36954 non-null  object 
 14  networks            33041 non-null  object 
 15  origin_country      44490 non-null  object 
 16  episode_

In [199]:
# Splitting the 'genres' feature by commas, only process if it's a string
df2['genres'] = df2['genres'].apply(lambda x: x.split(', ') if isinstance(x, str) else x)

# Removing leading/trailing spaces from each genres name and clean the genres names
df2['genres'] = df2['genres'].apply(lambda x: [genre.strip().lower().replace('-', '') for genre in x] if isinstance(x, list) else x)

# Flattening the list of language, getting the frequency count (counting each language)
genre_counts = df2['genres'].explode().value_counts()

# Getting the top 5 genres
top_5_genres = genre_counts.head(5).index

# Replacing genres not in the top 5 with 'Other' in the original column, but keep NaN as NaN
def group_genres(genres):
    if isinstance(genres, list):  # Only process if it's a list (after split)
        return [genre if genre in top_5_genres else 'Other' for genre in genres]
    else:  # Keep NaN values unchanged
        return genres

df2['genres'] = df2['genres'].apply(group_genres)

# Getting the frequency count and percentage of the grouped genres
grouped_genre_counts = df2['genres'].explode().value_counts()
grouped_genre_percentage = (grouped_genre_counts / len(df2)) * 100

# Combining the counts and percentages into a DataFrame
grouped_genre_summary = pd.DataFrame({
    'Count': grouped_genre_counts,
    'Percentage': grouped_genre_percentage
})

# Sorting by count and displaying the result
grouped_genre_summary = grouped_genre_summary.sort_values(by='Count', ascending=False)

# Printing the results
print("\nGrouped Genres (Top 5 + 'Other') Frequency and Percentage:")
print(grouped_genre_summary)

# Displaying the updated df2 with the modified 'genres' column
print("\nUpdated df2 with Grouped Genres:")
print(df2[['genres']].head())


Grouped Genres (Top 5 + 'Other') Frequency and Percentage:
             Count  Percentage
genres                        
Other        14787   28.784163
drama        11503   22.391575
documentary   9325   18.151912
comedy        6982   13.591061
reality       5447   10.603052
animation     3105    6.044149

Updated df2 with Grouped Genres:
                   genres
6   [Other, Other, drama]
9   [Other, Other, drama]
12         [drama, Other]
14  [Other, Other, drama]
16         [drama, Other]


In [200]:
# Handling "origin_country" column

# Checking for unique origin countries
unique_origin_countries = set(country for countries in df2['origin_country'] if isinstance(countries, list) for country in countries)

# Printing the number of unique origin countries and the actual countries
print(f"Total unique origin countries: {len(unique_origin_countries)}")
print(unique_origin_countries)

Total unique origin countries: 0
set()


In [201]:
# Splitting the 'origin_country' feature by commas, only process if it's a string
df2['origin_country'] = df2['origin_country'].apply(lambda x: x.split(',') if isinstance(x, str) else x)

# Removing leading/trailing spaces from each country name and cleaning the country names
df2['origin_country'] = df2['origin_country'].apply(lambda x: [country.strip().lower().replace('-', '') for country in x] if isinstance(x, list) else x)

# Flattening the list of countries and getting the frequency count (counting each country)
country_counts = df2['origin_country'].explode().value_counts()

# Getting the top 10 countries
top_10_countries = country_counts.head(10).index

# Replacing countries not in the top 10 with 'Other' in the original column, but keeping NaN as NaN
def group_countries(countries):
    if isinstance(countries, list):  # Only process if it's a list (after split)
        return [country if country in top_10_countries else 'Other' for country in countries]
    else:  # Keep NaN values unchanged
        return countries

df2['origin_country'] = df2['origin_country'].apply(group_countries)

# Getting the frequency count and percentage of the grouped countries
grouped_country_counts = df2['origin_country'].explode().value_counts()
grouped_country_percentage = (grouped_country_counts / len(df2)) * 100

# Combining the counts and percentages into a DataFrame
grouped_country_summary = pd.DataFrame({
    'Count': grouped_country_counts,
    'Percentage': grouped_country_percentage
})

# Sorting by count and display the result
grouped_country_summary = grouped_country_summary.sort_values(by='Count', ascending=False)

# Printing the results
print("\nGrouped Origin Countries (Top 10 + 'Other') Frequency and Percentage:")
print(grouped_country_summary)

# Displaying the updated df2 with the modified 'origin_country' column
print("\nUpdated df2 with Grouped Origin Countries:")
print(df2[['origin_country']].head())


Grouped Origin Countries (Top 10 + 'Other') Frequency and Percentage:
                Count  Percentage
origin_country                   
Other           15081   29.356459
us               7965   15.504555
cn               5085    9.898388
jp               3917    7.624776
kr               3271    6.367282
gb               2708    5.271354
de               1648    3.207973
in               1436    2.795297
fr               1433    2.789457
ca               1398    2.721327
br               1086    2.113992

Updated df2 with Grouped Origin Countries:
   origin_country
6            [kr]
9            [us]
12           [us]
14           [us]
16           [us]


Removing text features from df2 after copy them to df_text

In [203]:
columns_to_remove = ['name','overview'] 

# Copy the specified columns to df_text
df_text = df2[['name','overview']].copy()

# Remove the specified columns from df2
df2 = df2.drop(columns=columns_to_remove)

Data types convertion

In [205]:
df2[df2.select_dtypes(include=['object']).columns] = df2.select_dtypes(include=['object']).astype('string')

In [206]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 51372 entries, 6 to 168636
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   number_of_seasons   51372 non-null  int64  
 1   number_of_episodes  51372 non-null  int64  
 2   vote_count          51372 non-null  int64  
 3   vote_average        51372 non-null  float64
 4   adult               51372 non-null  bool   
 5   first_air_date      51372 non-null  string 
 6   last_air_date       50938 non-null  string 
 7   popularity          51372 non-null  float64
 8   type                51372 non-null  string 
 9   status              51372 non-null  string 
 10  genres              34485 non-null  string 
 11  languages           36954 non-null  string 
 12  networks            33041 non-null  string 
 13  origin_country      44490 non-null  string 
 14  episode_run_time    51372 non-null  int64  
dtypes: bool(1), float64(2), int64(4), string(8)
memory usage:

In [207]:
# Checking unique values in 'status' column
unique_status_count = df2['status'].nunique()
print(f"Unique values in 'status': {unique_status_count}")

# Checking unique values in 'type' column
unique_type_count = df2['type'].nunique()
print(f"Unique values in 'type': {unique_type_count}")

Unique values in 'status': 6
Unique values in 'type': 7


Pickle file

In [209]:
df2.to_pickle('df_after_data_prep.pkl')
df_text.to_pickle('text.pkl')